In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
df_data = pd.read_csv('../data/ogd32_elektrizitaetbilanz_jahreswerte.csv', sep=',', encoding='utf-16')
df_data.head(32)
df_data = df_data[df_data['Jahr'] == 2022]

df_data_erzeugung = df_data[[
    'Jahr', 
    'Erzeugung_laufwerk_GWh',
    'Erzeugung_speicherwerk_GWh',
    'Erzeugung_andere_erneuerbare_abfaelle_GWh',
    'Erzeugung_holz_GWh',
    'Erzeugung_biogas_GWh',
    'Erzeugung_photovoltaik_GWh',
    'Erzeugung_wind_GWh',
    'Erzeugung_andere_total_GWh',
    'Erzeugung_andere_fossil_GWh',
    'Erzeugung_kernkraftwerk_GWh',
]]

df_data_erzeugung = df_data_erzeugung[df_data['Jahr'] > 1970]

df_data_erzeugung.rename(columns={
    'Jahr': 'Year',
    'Erzeugung_laufwerk_GWh': 'Hydropower (run-of-river)',
    'Erzeugung_speicherwerk_GWh': 'Hydropower (storage)',
    'Erzeugung_andere_erneuerbare_abfaelle_GWh': 'Renewable waste',
    'Erzeugung_holz_GWh': 'Wood',
    'Erzeugung_biogas_GWh': 'Biogas',
    'Erzeugung_photovoltaik_GWh': 'Photovoltaic',
    'Erzeugung_wind_GWh': 'Wind',
    'Erzeugung_andere_total_GWh': 'Other (total)',
    'Erzeugung_andere_fossil_GWh': 'Fossil',
    'Erzeugung_kernkraftwerk_GWh': 'Nuclear power',
}, inplace=True)

df_data_erzeugung = df_data_erzeugung.drop('Other (total)', axis=1)

# Grouping columns
df_data_erzeugung_grouped = df_data_erzeugung.copy()

df_data_erzeugung_grouped['Nuclear'] = df_data_erzeugung_grouped['Nuclear power']
df_data_erzeugung_grouped['Fossil'] = df_data_erzeugung_grouped['Fossil']
df_data_erzeugung_grouped['Renewable'] = (
    df_data_erzeugung_grouped['Renewable waste'] +
    df_data_erzeugung_grouped['Wood'] +
    df_data_erzeugung_grouped['Biogas'] +
    df_data_erzeugung_grouped['Photovoltaic'] +
    df_data_erzeugung_grouped['Wind']+
    df_data_erzeugung_grouped['Hydropower (run-of-river)'] +
    df_data_erzeugung_grouped['Hydropower (storage)']
)

# Selecting only the grouped columns and the Year column
df_data_erzeugung_grouped = df_data_erzeugung_grouped[['Year', 'Nuclear', 'Fossil', 'Renewable']]

df_data_erzeugung_grouped.head()



,Year,Nuclear,Fossil,Renewable
62,2022,23113.0,878.0,39514.0


In [3]:


df_data = pd.read_csv('../data/EnergyMixEU.csv', sep='\t')
df_data.rename(columns={df_data.columns[0]: 'Country'}, inplace=True)

display(df_data) 


,Country,renewables,fossil fuels,nuclear
0,Luxembourg,92.80%,7.20%,0.00%
1,Denmark,79.31%,20.69%,0.00%
2,Latvia,75.51%,24.49%,0.00%
3,Lithuania,72.55%,27.45%,0.00%
4,Austria,69.44%,30.56%,0.00%
5,Sweden,69.32%,1.23%,29.45%
6,Croatia,64.64%,35.36%,0.00%
7,Portugal,62.67%,37.33%,0.00%
8,Finland,53.98%,11.00%,35.01%
9,Greece,46.34%,53.66%,0.00%


In [4]:
df_data["renewables"] = df_data["renewables"].str.rstrip("%").astype("float")
df_data["nuclear"] = df_data["nuclear"].str.rstrip("%").astype("float")
df_data["fossil fuels"] = df_data["fossil fuels"].str.rstrip("%").astype("float")


In [5]:
total_energy_ch = df_data_erzeugung_grouped['Renewable'].sum() + df_data_erzeugung_grouped['Nuclear'].sum() + df_data_erzeugung_grouped['Fossil'].sum()

def get_percentage(value):
    print(value, total_energy_ch)
    return (100 / total_energy_ch) * value

renewables_percentage = get_percentage(df_data_erzeugung_grouped['Renewable'].sum())
nuclear_percentage = get_percentage(df_data_erzeugung_grouped['Nuclear'].sum())
fossil_fuels_percentage = get_percentage(df_data_erzeugung_grouped['Fossil'].sum())


new_row = {
    "Country": "Switzerland",
    "renewables": renewables_percentage,
    "nuclear": nuclear_percentage,
    "fossil fuels": fossil_fuels_percentage
}

display(new_row)

df_data.loc[len(df_data)] = new_row


39514.0 63505.0
23113.0 63505.0
878.0 63505.0


{'Country': 'Switzerland',
 'renewables': np.float64(62.22187229352019),
 'nuclear': np.float64(36.39555940477128),
 'fossil fuels': np.float64(1.3825683017085268)}

In [6]:

df_data.columns.values[0] = "Country"
df_data.rename(columns={'fossil fuels': 'Fossil'}, inplace=True)
df_data.rename(columns={'renewables': 'Renewable'}, inplace=True)
df_data.rename(columns={'nuclear': 'Nuclear'}, inplace=True)

In [7]:
df_data = df_data.sort_values(by='Renewable', ascending=False)


display(df_data)

,Country,Renewable,Fossil,Nuclear
0,Luxembourg,92.800000,7.200000,0.000000
1,Denmark,79.310000,20.690000,0.000000
2,Latvia,75.510000,24.490000,0.000000
3,Lithuania,72.550000,27.450000,0.000000
4,Austria,69.440000,30.560000,0.000000
5,Sweden,69.320000,1.230000,29.450000
6,Croatia,64.640000,35.360000,0.000000
7,Portugal,62.670000,37.330000,0.000000
27,Switzerland,62.221872,1.382568,36.395559
8,Finland,53.980000,11.000000,35.010000


In [8]:
highcharts_series = []
for column in df_data.columns:
    series_data = {
        'name': column,
        'data': df_data[column].fillna(0).tolist()  # Replace NaN with 0 for Highcharts
    }
    highcharts_series.append(series_data) 
    
with open('../public/data/energiemix_eu.json', 'w') as json_file:
    json.dump(highcharts_series, json_file, indent=2)